In [49]:
from load_llm import openai_llm, groq_llm, llm
from langchain_core.prompts import ChatPromptTemplate

In [6]:
from typing import *
from langchain_core.pydantic_v1 import BaseModel, Field


class Supervisor(BaseModel):
    """select agent and explantion for selecting the agent"""

    agent: Literal["ai_expert", "stock_trader", "generic"] = Field(
        ...,
        description="""select following agents based user question agents: ai_expert, stock_trader, generic""",
    )

    explanation: str = Field(description="explain the reason for selecting specific agent")

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            "You are a supervisor tasked with selecting right agent to act next 
            
            ai_expert
            Expertise:  AI, Data science
            Description: An ai_researcher is an expert in researching and developing new machine learning and AI models, algorithms, and techniques. They have in-depth knowledge of data science and can collect, analyze, and interpret complex data to inform their research.

            stock_trader
            Expertise: Stock market analysis, trading
            Description: A stock_trader is an expert in analyzing and predicting stock market trends, and making informed trading decisions. They have in-depth knowledge of financial markets, economic indicators, and trading strategies.

            generic
            Expertise: General topics
            
            Description: If a topic does not fall under the specific expertise of the other agents, the query will default to the generic agent. This ensures that a wide array of questions can be addressed, even if they do not pertain to the specialized fields of the other agents.

            specify reason for selecting the agent
            """,
        ),
        ("human", "{query}"),
    ]
)

structured_llm = openai_llm.with_structured_output(Supervisor)

structured_llm = prompt | structured_llm

In [9]:
query = "How to learn spring framework"

res = structured_llm.invoke({"query": query})
print(res)

agent='generic' explanation='The question is about learning a specific software framework, which is a general topic and does not require specialized knowledge in AI or stock trading. Therefore, the generic agent is the most suitable to handle this query.'


In [10]:
res.agent

'generic'

In [7]:
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_community.retrievers import TavilySearchAPIRetriever


search_engine = DuckDuckGoSearchAPIWrapper()

In [10]:
print(search_engine._ddgs_text("Indian government statement on Hindenburg report", max_results=5)[0]["body"])

India's markets regulator on Sunday asked investors to remain calm and exercise due diligence before reacting to reports such as that by US-based shortseller Hindenburg Research.


In [6]:
docs

[[Document(metadata={'title': 'Elon Musk | Tesla', 'source': 'https://www.tesla.com/elon-musk', 'score': 0.9964064, 'images': []}, page_content="Learn about Elon Musk's vision and achievements as the leader of Tesla, SpaceX, Neuralink and The Boring Company. Tesla is a sustainable energy company that produces electric vehicles, battery products and solar power systems, while SpaceX is a space exploration company that aims to colonize Mars."),
  Document(metadata={'title': 'Who Is Elon Musk? - Investopedia', 'source': 'https://www.investopedia.com/articles/personal-finance/061015/how-elon-musk-became-elon-musk.asp', 'score': 0.98439646, 'images': []}, page_content="“Elon Musk Tried to Pitch the Head of the Yellow Pages Before the Internet Boom: ‘He Threw the Book at Me’.”\nCompaq Computer, via U.S. Securities and Exchange Commission. SpaceX\nMusk used most of the proceeds from his PayPal stake to found Space Exploration Technologies Corp., the rocket's developer commonly known as SpaceX

In [50]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

tools = [TavilySearchResults(max_results=1)]

In [82]:
tools[0].invoke("rohit sharma")

[{'url': 'https://www.bbc.co.uk/news/world-asia-india-67482722',
  'content': 'BBC News, Delhi. Millions of Indians cried along with cricket captain Rohit Sharma on Sunday as he walked off the field, trying to hold back his tears. His team had just lost the cricket World Cup ...'}]

In [80]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information. Give explanation why you are using a tool",
        ),
        ("placeholder", "{chat_history}"),
        ("placeholder", "{agent_scratchpad}"),
        ("human", "{input}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_intermediate_steps=True
)

In [28]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
res = agent_executor.invoke({"input": "what is LangChain?"})

In [29]:
res.keys()

dict_keys(['input', 'output'])

In [31]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are a helpful assistant.
            """,
        ),
        ("placeholder", "{chat_history}"),
    ]
)

In [33]:
from langchain_core.messages import *

value = prompt.invoke({"chat_history": [HumanMessage(content="hi"), AIMessage(content="hello")]})

In [34]:
value.messages

[SystemMessage(content='\nYou are a helpful assistant.\n            '),
 HumanMessage(content='hi'),
 AIMessage(content='hello')]

In [62]:
from load_llm import openai_llm, groq_llm
from langchain_core.prompts import ChatPromptTemplate

from typing import *
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import *


class Planner(BaseModel):
    """list of steps to answer the query"""

    steps: List[str] = Field(
        ...,
        description="""list of steps to answer the query""",
    )


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
As a planner, list all the steps required to answer the query
            Previous conversation history:

            {chat_history}
            """,
        ),
        ("human", "{query}"),
    ]
)

structured_llm = groq_llm.with_structured_output(Planner)

structured_llm = prompt | structured_llm


# # Create an agent executor by passing in the agent and tools
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [78]:
query = """
why osama bin laden destroyed twin towers """
response = structured_llm.invoke({"query": query, "chat_history": []})

plan = response.steps
plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
print(plan_str)

1. 1. Identify the motivations behind Osama bin Laden's actions
2. 2. Research the events leading up to the 9/11 attacks
3. 3. Understand the role of Al-Qaeda in the attacks
4. 4. Analyze the impact of the attacks on global politics and international relations
5. 5. Evaluate the consequences of the attacks on the United States and the world


In [81]:
input = f"""
For below user query
{query} 
Use below points while responding: 
{plan_str}
                      """

res = agent_executor.invoke({"input": input})
from IPython.display import display, Markdown, Latex

display(Markdown(res["output"]))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Osama bin Laden motivations for 9/11 attacks'}`


[{'url': 'https://en.wikipedia.org/wiki/Motives_for_the_September_11_attacks', 'content': 'In an essay, Doran argued that bin Laden\'s followers "consider themselves an island of true believers surrounded by a sea of iniquity".[65] Doran further argued that bin Laden hoped that US retaliation would unite the faithful against the West, sparking revolutions in Arab nations and elsewhere, and that the Osama bin Laden videos were attempting to provoke a visceral reaction in the Middle East aimed at a violent reaction by Muslim citizens to increased US involvement in their region.[66]\nBergen argued that the attacks were part of a plan to cause the United States to increase its military and cultural presence in the Middle East, thereby forcing Muslims to confront the idea of a non-Muslim government and establish conservative Islamic governments i

Osama bin Laden destroyed the Twin Towers as part of a larger plan to inflict harm on the United States and its interests. The motivations behind his actions were rooted in his extremist ideology and a desire to defend Islam against perceived Western aggression.

Bin Laden was motivated by a range of factors, including:

1. Opposition to US foreign policy in the Middle East, particularly the presence of US troops in Saudi Arabia.
2. Anger over US support for Israel and its treatment of Palestinians.
3. A desire to unite Muslims against the West and establish a global Islamic caliphate.

The events leading up to the 9/11 attacks were complex and multifaceted. They included:

1. The Soviet-Afghan War, which drew bin Laden and other jihadists to Afghanistan to fight against the Soviet Union.
2. The Gulf War, which led to the presence of US troops in Saudi Arabia and further radicalized bin Laden.
3. The rise of Al-Qaeda, which bin Laden founded in the late 1980s and which became a global network of jihadists.

Al-Qaeda played a central role in the 9/11 attacks, which were planned and carried out by a group of operatives led by bin Laden. The attacks were intended to inflict maximum damage on the United States and its economy, and to create a sense of fear and uncertainty among the American people.

The impact of the 9/11 attacks on global politics and international relations was significant. They led to:

1. The launch of the War on Terror, a global military campaign aimed at rooting out Al-Qaeda and other jihadist groups.
2. The invasion of Afghanistan, which toppled the Taliban regime and led to a long and bloody conflict.
3. The invasion of Iraq, which was justified by the US government as part of the War on Terror but was widely criticized as a mistake.

The consequences of the 9/11 attacks on the United States and the world were far-reaching. They included:

1. The loss of thousands of lives, both on the day of the attacks and in the subsequent wars in Afghanistan and Iraq.
2. A significant increase in military spending and a shift towards a more militarized foreign policy.
3. A rise in Islamophobia and anti-Muslim sentiment in the United States and other Western countries.
4. A decline in civil liberties and an increase in surveillance and security measures.

Overall, the 9/11 attacks were a pivotal moment in modern history, with far-reaching consequences for global politics, international relations, and the lives of millions of people around the world.

In [69]:
response = groq_llm.invoke(
    f"""
For below user query
{query} 
Use below points while responding: 
{plan_str}

                      """
)
from IPython.display import display, Markdown, Latex

display(Markdown(response.content))

The 2008 market crash, also known as the Global Financial Crisis (GFC), was a complex and multifaceted event with various contributing factors. Here's a breakdown of the key events and factors that led to the crisis:

**Key Events Leading Up to the 2008 Market Crash:**

1. **Housing Market Boom (2000-2006)**: The US housing market experienced a significant boom, fueled by low interest rates, lax lending standards, and excessive speculation.
2. **Subprime Mortgage Crisis (2006-2007)**: Many subprime borrowers, who were not able to afford mortgages, defaulted on their loans, causing a surge in foreclosures and a subsequent decline in housing prices.
3. **Financial Deregulation (1999-2004)**: The Gramm-Leach-Bliley Act (1999) repealed parts of the Glass-Steagall Act, allowing commercial banks to engage in investment activities, increasing their exposure to risk. The Securities and Exchange Commission (SEC) also relaxed regulations, enabling financial institutions to take on more leverage.
4. **Excessive Leverage and Risk-Taking (2004-2007)**: Financial institutions, including banks and investment banks, took on excessive leverage and risk, creating complex financial instruments, such as mortgage-backed securities (MBS) and collateralized debt obligations (CDOs).

**Role of Subprime Mortgage Crisis:**

The subprime mortgage crisis played a significant role in the 2008 market crash. Many subprime borrowers were unable to afford their mortgages, leading to a wave of defaults and foreclosures. This caused a decline in housing prices, which in turn led to a decline in the value of mortgage-backed securities (MBS) and collateralized debt obligations (CDOs) held by financial institutions.

**Impact of Housing Market Bubble Burst:**

The housing market bubble burst in 2006-2007, leading to a sharp decline in housing prices. This decline in housing prices reduced the value of mortgage-backed securities (MBS) and collateralized debt obligations (CDOs), causing significant losses for financial institutions.

**Effect of Financial Deregulation:**

Financial deregulation contributed to the 2008 market crash by allowing financial institutions to take on excessive leverage and risk. The Gramm-Leach-Bliley Act (1999) and the SEC's relaxation of regulations enabled commercial banks to engage in investment activities, increasing their exposure to risk.

**Consequences of Excessive Leverage and Risk-Taking:**

Excessive leverage and risk-taking by financial institutions led to a significant increase in their exposure to risk. When the housing market bubble burst, many financial institutions found themselves with large amounts of worthless mortgage-backed securities (MBS) and collateralized debt obligations (CDOs), leading to significant losses and eventual collapse.

**Timeline of Major Events During the Crisis:**

* August 2007: French bank BNP Paribas halts withdrawals from its investment funds, citing subprime mortgage exposure.
* March 2008: Bear Stearns, a US investment bank, is sold to JPMorgan Chase after facing financial difficulties.
* September 7, 2008: Fannie Mae and Freddie Mac, US government-sponsored mortgage giants, are placed into conservatorship.
* September 14, 2008: Merrill Lynch, a US investment bank, is sold to Bank of America.
* September 15, 2008: Lehman Brothers, a US investment bank, files for bankruptcy.
* October 3, 2008: The US government passes the Troubled Asset Relief Program (TARP) to provide $700 billion in bailout funds to struggling financial institutions.

**Key Factors That Contributed to the 2008 Market Crash:**

1. **Subprime Mortgage Crisis**: Widespread defaults on subprime mortgages led to a decline in housing prices and a subsequent decline in the value of mortgage-backed securities (MBS) and collateralized debt obligations (CDOs).
2. **Financial Deregulation**: The Gramm-Leach-Bliley Act (1999) and the SEC's relaxation of regulations enabled financial institutions to take on excessive leverage and risk.
3. **Excessive Leverage and Risk-Taking**: Financial institutions took on excessive leverage and risk, creating complex financial instruments that ultimately led to significant losses.
4. **Housing Market Bubble Burst**: The housing market bubble burst in 2006-2007, leading to a sharp decline in housing prices and a subsequent decline in the value of mortgage-backed securities (MBS) and collateralized debt obligations (CDOs).

In summary, the 2008 market crash was caused by a combination of factors, including the subprime mortgage crisis, financial deregulation, excessive leverage and risk-taking, and the housing market bubble burst. These factors created a perfect storm that ultimately led to the collapse of several major financial institutions and a global economic downturn.

In [89]:
from langchain_community.retrievers import TavilySearchAPIRetriever

tavily_retriever = TavilySearchAPIRetriever(k=2)



def get_results_from_tavily(search_queries):
    queries_docs = tavily_retriever.batch(search_queries)
    docs = [{"href": doc.metadata["source"], "content": doc.page_content} for docs in queries_docs for doc in docs]
    return docs


In [90]:
def search_web(search_queries: str, explanation: str) -> str:
    """The search_web tool allows to augment knowledge and retrieve the latest information on a topic by searching the web using provided comma-separated search queries max queries 2. This tool returns a summary of the key points from all retrieved web pages, enabling me to provide more accurate and up-to-date information.

    Args:
        search_queries (str): comma seperated search queries without quotes, related to the topic, max queries 3
        explanation (str): give a reason for choosing this tool

    Returns:
        str: summary of all web pages retrieved for a search query
    """

    search_queries = search_queries.split(",")
    # search_queries = [clean_text(query) for query in search_queries]

    urls = []
    results = get_results_from_tavily(search_queries)
    urls += [item["href"] for item in results]

    print("****   WEB SEARCH  *****")
    print(search_queries)
    for url in urls:
        print(url)

    summaries = "\n\n".join([f"\n\nContent: {(doc['content'])}" for idx, doc in enumerate(results)])
    return summaries

In [96]:
print(search_web("elon musk wikipedia, jeff bezos", ""))

****   WEB SEARCH  *****
['elon musk wikipedia', ' jeff bezos']
https://www.wikiwand.com/en/Elon_Musk
https://simple.wikipedia.org/wiki/Elon_Musk
https://www.britannica.com/money/Jeff-Bezos
https://en.wikipedia.org/wiki/Jeff_Bezos


Content: He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023[update], according to the Bloomberg Billionaires Index, and $254 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.[5][6]
A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. He is the founder, chairman, CEO, and chief technology officer of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and presiden